## Growth Challenge @ The Information

Explanation
For the purposes of this challenge, assume The Information is experimenting with new types of subject lines for our article emails to increase performance. Thus far, five articles have been sent since the experiment started. Two of these articles contain normal subject (article title), two have slight variations from normal subject lines, and the last has a wildly experimental subject. 

The accompanying spreadsheet contains the outlines of two tables: email_events and articles. The relationship between the two tables is a one-to-many in that articles can have many email_events while an email can only be associated with one article. Please note, an article email that was opened will have both a delivered and an open event.

In [167]:
import pandas as pd
import seaborn as sns
from sqlalchemy import create_engine
from pprint import pprint

In [3]:
email_events = pd.read_excel('Growth_Analyst_Challenge.xlsx', sheet_name='Email_events')
articles = pd.read_excel('Growth_Analyst_Challenge.xlsx', sheet_name='Articles')

In [7]:
print(email_events.shape)
email_events.head(3)

(51548, 3)


,id,article_id,action
0,1,5,delivered
1,2,1,open
2,3,1,delivered


In [184]:
print(articles.shape)
articles.head()

(5, 3)


,id,title,subject
0,1,"Bitcoin, Unicorns and Why All Bubbles Aren’t C...",hey… you should read this
1,2,Google May Step Into Gaming With ‘Yeti’ Stream...,EXCLUSIVE: Google May Step Into Gaming With ‘Y...
2,3,Spotify’s Q3 Revenue Points to Full Year Growt...,Exclusive: Spotify’s Q3 Revenue Points to Full...
3,4,Why 2021-2022 Could Be a Watershed for TV Sports,Why 2021-2022 Could Be a Watershed for TV Sports
4,5,"At Tencent, Tensions Rise Over the Future of W...","At Tencent, Tensions Rise Over the Future of W..."


In [28]:
merged = pd.merge(left=email_events, right=articles, how='left', left_on='article_id', right_on='id')
print(merged.shape)
merged = merged.drop(['id_y'], axis=1).rename(columns={"id_x":"id"})
print(merged.shape)

(51548, 6)
(51548, 5)


In [29]:
merged.head(3)

,id,article_id,action,title,subject
0,1,5,delivered,"At Tencent, Tensions Rise Over the Future of W...","At Tencent, Tensions Rise Over the Future of W..."
1,2,1,open,"Bitcoin, Unicorns and Why All Bubbles Aren’t C...",hey… you should read this
2,3,1,delivered,"Bitcoin, Unicorns and Why All Bubbles Aren’t C...",hey… you should read this


## Question 1

**Management has asked to see a report with the preliminary results of the experiment. Specifically, they would like to see which articles / subject lines have performed the best in terms of open rate, click rate, and click-to-open rate. By using the tool of your choice, please provide management with an analysis of the results from the first five emails.**

In [30]:
merged['action'].value_counts()

delivered    29640
open         16521
click         5387
Name: action, dtype: int64

In [65]:
# # Method through groupby > value_counts
# action=pd.DataFrame(merged.groupby('article_id')['action'].value_counts())

In [78]:
# Using pivot table
pivot = merged.pivot_table(index='article_id', columns='action', aggfunc='size')

In [77]:
pivot

action,click,delivered,open
article_id,,,
1,1180,5938,4056
2,1188,5930,3356
3,1037,5920,3122
4,989,5926,3015
5,993,5926,2972


In [82]:
open_rate = round(pivot['open']/pivot['delivered'] * 100, 2)
click_rate = round(pivot['click']/pivot['delivered'] * 100, 2)
click_to_open_rate = round(pivot['click']/pivot['open'] * 100, 2)

In [88]:
pivot['open_rate'] = open_rate
pivot['click_rate'] = click_rate
pivot['click_to_open_rate'] = click_to_open_rate

In [176]:
pivot = pivot.reset_index()

In [182]:
pivot_merge = pd.merge(left=articles, right=pivot, how='left', left_on='id', right_on='article_id').drop(['id'], axis=1)

In [183]:
pivot_merge

,title,subject,article_id,click,delivered,open,open_rate,click_rate,click_to_open_rate
0,"Bitcoin, Unicorns and Why All Bubbles Aren’t C...",hey… you should read this,1,1180,5938,4056,68.31,19.87,29.09
1,Google May Step Into Gaming With ‘Yeti’ Stream...,EXCLUSIVE: Google May Step Into Gaming With ‘Y...,2,1188,5930,3356,56.59,20.03,35.40
2,Spotify’s Q3 Revenue Points to Full Year Growt...,Exclusive: Spotify’s Q3 Revenue Points to Full...,3,1037,5920,3122,52.74,17.52,33.22
3,Why 2021-2022 Could Be a Watershed for TV Sports,Why 2021-2022 Could Be a Watershed for TV Sports,4,989,5926,3015,50.88,16.69,32.80
4,"At Tencent, Tensions Rise Over the Future of W...","At Tencent, Tensions Rise Over the Future of W...",5,993,5926,2972,50.15,16.76,33.41


## Question 2

**Please provide management with some suggestions on how they could scale or change this test to get a better understanding of the effectiveness of the changes being tested.**

1. I would add another table 'users' to identify which type of users (paid/free) are opening / clicking the emails.
 - Are the users subscribing/unsubscribing/marking-as-spam after engaging with the email? (subscribe/unsubscribe rate, conversion rate).
 - Is the user sharing/forwarding the email to other people?
 
2. I would add a time variable that accounts for how long the user engages with the email 
 - how long it keeps the email open?
 - how long does it take before the user opens/clicks the email?
 
3. Maybe we are seeing larger ratios for some sent emails because they were sent during better times
 - We might want to send all tests at the same times
 - What time is the most effective in terms of opening and clicking each email test?
 
4. Device type?
 - Is the user opening/clicking the emails on a tablet/mobile instead of a desktop? Is the user opening on a mobile and then using a desktop? This might indicate that the user got interested on the email and is now spending more time engaging with it using a bigger display.
 
5. We might also want to analyze/compare the most efficient subject line lengths.
 - What is the optimal character length?


## Question 3

**As a thought exercise, assume the number of email_events being generated is now on the order of ~3M per day. What strategies, tools, or technologies would you advise management that we should be using in order to effectively and performantly analyze the data?**

I believe that we are using a cloud storage service. (AWS S3, redshift/ GCP)
We might want to build a data pipeline (ETL) that performs scheduling and monitoring of data analysis (weekly/monthly) and records dashboard updates for each time period, comparing the email experiments.
 - The ETL (e.g. Apache Spark) retrieves data from the warehouse periodically, transforms it, and connects to the dashboard service.


## Question 4

**Please construct a SQL statement using the two tables (in the spreadsheet, just assume they’re sql tables) with the following columns in the output ordered by the open rate from highest to lowest:**

I answered this questions using Jupyter Notebook (Python > SQLAlchemy (SQLite) and also using PSQL (pgAdmin4_

In [97]:
engine = create_engine('sqlite://', echo=False)

In [98]:
email_events.to_sql('email_events', con=engine)
articles.to_sql('articles', con=engine)

In [162]:
query = engine.execute("SELECT article_id, \
subject, \
title, \
ROUND(CAST(open as float) / \
      CAST(delivered as float)*100, 2 \
      ) AS open_rate, \
click, \
delivered, \
open \
FROM \
    (SELECT article_id, \
        COUNT(CASE WHEN action = 'delivered' THEN action END) delivered, \
        COUNT(CASE WHEN action = 'click' THEN action END) click, \
        COUNT(CASE WHEN action = 'open' THEN action END) open FROM email_events \
    GROUP BY article_id \
    ) AS pivot \
JOIN articles ar ON ar.id = pivot.article_id ORDER BY open_rate DESC").fetchall() 

In [166]:
for i in query:
    print(i)

(1, 'hey… you should read this', 'Bitcoin, Unicorns and Why All Bubbles Aren’t Created Equal', 68.31, 1180, 5938, 4056)
(2, 'EXCLUSIVE: Google May Step Into Gaming With ‘Yeti’ Streaming Service', 'Google May Step Into Gaming With ‘Yeti’ Streaming Service', 56.59, 1188, 5930, 3356)
(3, 'Exclusive: Spotify’s Q3 Revenue Points to Full Year Growth of 40%', 'Spotify’s Q3 Revenue Points to Full Year Growth of 40%', 52.74, 1037, 5920, 3122)
(4, 'Why 2021-2022 Could Be a Watershed for TV Sports', 'Why 2021-2022 Could Be a Watershed for TV Sports', 50.88, 989, 5926, 3015)
(5, 'At Tencent, Tensions Rise Over the Future of WeChat', 'At Tencent, Tensions Rise Over the Future of WeChat', 50.15, 993, 5926, 2972)


<img src="Screen Shot 2021-04-25 at 1.20.49 PM.png">